In [9]:
# Author: M. Riley Owens (GitHu: mrileyowens)

# This file determines how the Lyman-alpha flux
# in the MagE apertures changes due to seeing effects,
# as estimated from the narrowband Lyman-alpha
# images.

In [10]:
import os
import glob

import numpy as np

from astropy.io import fits
from astropy.convolution import convolve, convolve_fft, Gaussian2DKernel
from astropy import uncertainty as unc

import matplotlib.pyplot as plt

In [11]:
def func():
    
    home = os.getcwd()
    data = f'{home}/data'
    results = f'{home}/results'
    
    slits = np.array(['M5', 'M4', 'M6', 'M3', 'M0', 'M2', 'M7', 'M8', 'M9'], 
        dtype=str)
    
    # Dictionary of roughly-estimated time-weighted seeing conditions, 
    # written as the FWHM of the PSF in arcseconds
    seeing = {
        'M5' : 0.97,
        'M4' : 0.71,
        'M6' : 0.76,
        'M3' : 0.70,
        'M0' : 1.34,
        'M2' : 0.77,
        'M7' : 0.73,
        'M8' : 0.70,
        'M9' : 0.68
    }

    # Dictionary of the airmasses of the individual slit pointings,
    # written as sec(z)
    airmass = {
        'M5' : [1.756, 1.668, 1.602, 1.553, 1.531],
        'M4' : [1.529, 1.532, 1.549],
        'M6' : [1.530, 1.549, 1.581, 1.531, 1.552],
        'M3' : [1.775, 1.690, 1.627, 1.571, 1.542],
        'M0' : [1.540, 1.528, 1.536, 1.565, 1.656],
        'M2' : [1.584, 1.639, 1.592],
        'M7' : [1.563, 1.600, 1.665, 1.569, 1.610, 1.672],
        'M8' : [1.528, 1.537, 1.747, 1.529, 1.539, 1.757],
        'M9' : [1.738, 1.654, 1.592, 1.552, 1.531]
    }

    # Dictionary of the exposure times of the individual slit pointings,
    # written in seconds
    exposures = {
        'M5' : [2700, 2700, 2700, 2700, 2700],
        'M4' : [2400, 2400, 2400],
        'M6' : [2700, 2400, 2400, 2700, 2700],
        'M3' : [2400, 2400, 2400, 2400, 2400],
        'M0' : [2700, 2700, 2700, 2700, 2700],
        'M2' : [2700, 2700, 2700],
        'M7' : [2400, 2400, 2400, 2400, 2400, 2400],
        'M8' : [2400, 2400, 2400, 2400, 2400, 2400],
        'M9' : [2700, 2700, 2700, 2700, 2700]       
    }

    #lya_files = glob.glob(f'{data}/hst/Lya_contsub*.fits')
    filters = np.array(['F390W', 'F555W'], dtype=str)

    for i, slit in enumerate(slits):
    
        slit_mask_file = f'{results}/{slit}_mask_v5.fits'

        slit_mask = fits.getdata(slit_mask_file)[4000:5600, 4400:5700]

        for j, filter in enumerate(filters):

            file = f'{results}/Lya_cont_sub_{filter}.fits'

            lya = fits.open(file)[0].data[4000:5600, 4400:5700]
            #mask = fits.open(f'{results}/box_for_median_imcoords_mask_v5.fits')[0].data[4300:5300, 4700:5400]
            n = fits.open(file)[1].data[4000:5600, 4400:5700]
            
            n = np.sqrt(np.sum(np.square(n * slit_mask)))

            #samples = np.ones_like(lya) * 1000

            #lya = unc.Distribution(np.append(lya, samples, axis=2))
            #lya = unc.normal(lya, std=n, n_samples=1000)

            #stddev = ((0.70 * (1.641)**0.6) / 2.355) / 0.03

            # The standard deviation of the PSF in pixels
            stddev = ((seeing[slit] * (np.sum(np.multiply(airmass[slit], exposures[slit]) / np.sum(exposures[slit])))**0.6) / 2.355) / 0.03

            kernel = Gaussian2DKernel(x_stddev=stddev, y_stddev=stddev)

            lya_convolved = convolve_fft(lya, kernel)
            n_convolved = np.std(lya_convolved * slit_mask)

            flux = np.sum(mask * lya)
            flux_convolved = np.sum(mask * lya_convolved)

            print(file, slit)
            # Should this be an absolute value?
            print(flux / flux_convolved)

In [12]:
func()

(1600, 1300)
c:\Users\15136\OneDrive\Documents\Research\sunburstarc/results/Lya_cont_sub_F390W.fits M5
0.948150567616979
(1600, 1300)
c:\Users\15136\OneDrive\Documents\Research\sunburstarc/results/Lya_cont_sub_F555W.fits M5
0.6834097868783173
(1600, 1300)
c:\Users\15136\OneDrive\Documents\Research\sunburstarc/results/Lya_cont_sub_F390W.fits M4
0.9471816274918697
(1600, 1300)
c:\Users\15136\OneDrive\Documents\Research\sunburstarc/results/Lya_cont_sub_F555W.fits M4
0.11300822458583029
(1600, 1300)
c:\Users\15136\OneDrive\Documents\Research\sunburstarc/results/Lya_cont_sub_F390W.fits M6
0.9270659967667306
(1600, 1300)
c:\Users\15136\OneDrive\Documents\Research\sunburstarc/results/Lya_cont_sub_F555W.fits M6
0.609542430899161
(1600, 1300)
c:\Users\15136\OneDrive\Documents\Research\sunburstarc/results/Lya_cont_sub_F390W.fits M3
0.941462029162151
(1600, 1300)
c:\Users\15136\OneDrive\Documents\Research\sunburstarc/results/Lya_cont_sub_F555W.fits M3
-0.9160722783463432
(1600, 1300)
c:\Users\151